#### 开头

In [ ]:
import pandas as pd
import xlwings as xw
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#### 小知识

> 进度条  

> 主线程通过queue推送数据，进度条通过获取总数量和主线程que数据计算进度

In [ ]:
from tqdm import tqdm
from time import sleep
from multiprocessing.dummy import Process
from queue import Queue


def f(x):
    for eh in x:
        que.put(eh)  # 处理完一个结果后，放到queue中
        sleep(1)

def jdt(mylist):
    for _ in (bar := tqdm(mylist)):
        while que.empty():
            sleep(.1)
        bar.set_description(f'正在处理 {que.get()}')

que = Queue()
mylist = list('12345678')
t = Process(target=jdt, args=(mylist,))
t.start()
f(mylist)
t.join()


> 处理pandas的apply数据加个进度条

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.DataFrame(np.random.randint(0, 100, (10000000, 100)))
tqdm.pandas(desc="my bar！")
df.progress_apply(lambda x:x*3,) # 替换apply

> 自定义季度

In [ ]:
pd.Period('2022-11-30', freq='Q-Nov').quarter
pd.Period('2022-12-1', freq='Q-Nov').quarter
pd.Period('2022-2-28', freq='Q-Nov').quarter
pd.Period('2022-3-3', freq='Q-Nov').quarter


> 持续读取文本内容，适合于类似日志不断增加的情况

In [ ]:
import time


def rdtxt_conti(filename):
    with open(filename, 'r') as f:
        yield from f.readlines()
        f.seek(0, 2)
        while 1:
            if (txt := f.readline()):
                yield txt
            else:
                time.sleep(4)


if __name__ == '__main__':
    '''持续读取文本内容，适合于类似日志不断增加的情况'''
    for x in rdtxt_conti('test.txt'):
        print(x)


> 字段名重名的自动扩展，wide to long

In [ ]:
def first_w_l(df, col_n_list: list = None):
    '''
    字段名重名的自动扩展，wide to long
    默认按照第一列为扩展依据。
    如果有多列，按照列的顺序组成列表作为第二参数传递，比如col_n_list=[0,1]
    '''
    if col_n_list is None:
        col_n_list = [0]
    i = df.columns[col_n_list].tolist()
    df = df.set_index(i)
    stubnames = df.filter(regex=r'^[^.]+$').columns
    repdic = {x: f'{x}.0' for x in stubnames}
    df = df.reset_index().rename(columns=repdic)
    return (pd.wide_to_long(df, stubnames=stubnames, i=i, j='n', sep='.')
            .sort_index(level=-1)
            .droplevel(-1))


df = pd.read_excel('test.xlsx')
df
first_w_l(df, [0, 1])


> 时间格式

In [ ]:
import time
time.strftime('%F %T')
'''
%a 星期几的简写
%A 星期几的全称
%b 月分的简写
%B 月份的全称
%c 标准的日期的时间串
%C 年份的后两位数字
%d 十进制表示的每月的第几天
%D 月/天/年
%e 在两字符域中，十进制表示的每月的第几天
%F 年-月-日
%g 年份的后两位数字，使用基于周的年
%G 年分，使用基于周的年
%h 简写的月份名
%H 24小时制的小时
%I 12小时制的小时
%j 十进制表示的每年的第几天
%m 十进制表示的月份
%M 十时制表示的分钟数
%n 新行符
%p 本地的AM或PM的等价显示
%r 12小时的时间
%R 显示小时和分钟：hh:mm
%S 十进制的秒数
%t 水平制表符
%T 显示时分秒：hh:mm:ss
%u 每周的第几天，星期一为第一天 （值从0到6，星期一为0）
%U 第年的第几周，把星期日做为第一天（值从0到53）
%V 每年的第几周，使用基于周的年
%w 十进制表示的星期几（值从0到6，星期天为0）
%W 每年的第几周，把星期一做为第一天（值从0到53）
%x 标准的日期串
%X 标准的时间串
%y 不带世纪的十进制年份（值从0到99）
%Y 带世纪部分的十进制年份
%z %Z 时区名称，如果不能得到时区名称则返回空字符。
%% 百分号'''


> 上下文管理器

In [ ]:
from contextlib import contextmanager
import time


def myfp(x):
    print(x)
    time.sleep(1)


@contextmanager
def withf():
    st = time.perf_counter()
    print('first')
    yield myfp
    print('used_time:', time.perf_counter()-st)


with withf() as k:
    k('aab')


如果一个对象没有实现上下文，我们就不能把它用于with语句。这个时候，可以用closing()来把该对象变为上下文对象。例如，用with语句使用urlopen()：

In [ ]:
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('https://www.python.org')) as page:
    for line in page:
        print(line)


> 一维转二维

In [ ]:
n = 3
se = pd.Series([1, 2, 3, 4, 5, 6, 7])
se = pd.Series([1, 2, 3, 4, 5, 6, 7]*100000)
(se.reset_index(drop=True)
    .to_frame()
    .set_axis(tuple(zip(se.index//n, se.index % n)))
    .unstack(1,)
    .droplevel(0, axis=1)
 )


In [ ]:
se = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
se = pd.Series([1, 2, 3, 4, 5, 6, 7]*100000)
n = 3
se = np.concatenate([se.values, np.repeat([np.nan], n)])
pd.DataFrame(se[:len(se)//n*n].reshape((len(se)//n, -1))).dropna(how='all')


In [ ]:
se = pd.Series([1, 2, 3, 4, 5, 6, 7])
se = pd.Series([1, 2, 3, 4, 5, 6, 7]*100000)
se.groupby(se.index//3).agg(list).apply(pd.Series)
'''效率比较低,不推荐'''


> os.walk获取全文件名

In [ ]:
import os
mypath = r'D:\netdisk\learn\Python'
# for  root,dir,files in os.walk(mypath):
#     for ehname in files:
#         print(os.path.join(root,ehname))

filelist = [os.path.join(root, ehname) for root, _, files in os.walk(mypath)
            for ehname in files if ehname.endswith('.py')]
# xw.view(pd.DataFrame(filelist,columns=['文件名']))
'''如果获取目录下文文件，只要用os.listdir()'''
[os.path.join(mypath, x) for x in os.listdir(mypath)]


> 定位1-2之间的索引,定位begin和end之间的数据   
> 学习 pd.merge_asof，类似match的匹配模式

In [ ]:
'''定位1-2之间的索引,定位begin和end之间的数据'''
df = pd.DataFrame(
    {'a': [0, 1, 2, 0, 1, 0, 0, 2, 0, 1, 0, 0, 2, 0, 1]}).reset_index()
df = pd.DataFrame(
    {'a': [2, 1, 1, 2, 1, 1, 10, 0, 2, 0, 1, 0, 1, 2, 2, 2]}).reset_index()

ind = (pd.merge_asof(
    df.query('a==1').rename(columns={'index': 'begin'}),
    df.query('a==2').rename(columns={'index': 'end'}),
    left_on='begin', right_on='end',
    direction='forward',)
    .dropna().astype(int)[['begin', 'end']]
    # 控制组范围，比如：1102，last为102、first为1102
    .drop_duplicates(subset='end', keep='last')
    .apply(lambda x: list(range(x[0], x[1]+1)), axis=1)
    .sum()
)
ind
df['r'] = df['a'].mask(df.index.isin(ind), 9)
# df.iloc[ind,-1]=9
df


In [ ]:
df = pd.DataFrame({'a': [2, 1, 1, 2, 1, 1, 10, 0, 2, 0, 1, 0, 1, 2, 2, 2]})
g1 = df['a'].eq(1).cumsum()
g2 = df['a'].eq(2)[::-1].cumsum()
m = df.groupby([g1, g2])['a'].transform(
    lambda g: g.iloc[0] == 1 and g.iloc[-1] == 2)
df['desired'] = pd.Series(9, index=m.index).mask(~m, df['a'])
df.assign(g1=g1, g2=g2, m=m)


> 获取网页最新汇率

In [ ]:
url = 'https://fx.cmbchina.com/hq/'  # 网站名
df = pd.read_html(url, encoding='utf-8', match=r'交易币',
                  header=0, index_col='交易币')[0]
df


> str.translate  单字符映射表

In [ ]:
transtable = str.maketrans('abc', '123')
'abcdefacdb'.translate(transtable)
bytes_tabtrans = bytes.maketrans(
    b'abcdefghijklmnopqrstuvwxyz', b'ABCDEFGHIJKLMNOPQRSTUVWXYZ')
b'runoob'.translate(bytes_tabtrans, b'o')


> filter 连顺序都排列好了，比reindex好用

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [3, 4, 5]})
df
# df.reindex(['a','c'],axis=1).dropna(how='all',axis=1)
df.filter(['c', 'b', 'a'])


> 字典推导式


In [ ]:
in_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 8, 7]
print('array before:', in_list)
array = {i: i * 2 for i in in_list if i % 2 != 0}  # 字典推导表达式
print('array after:', array)


> 百家姓 正则

In [ ]:
import re
s = '张三 zxcxzcxzcxz 李四 出自行车正常 周杰伦 沈腾'
xin1 = '[王李张刘陈杨黄赵吴周徐孙马朱胡郭何高林罗郑梁谢宋唐许韩冯邓曹彭曾肖田董袁潘于蒋蔡余杜叶程苏魏吕丁任沈姚卢姜崔钟谭陆汪范金石廖贾夏韦傅方白邹孟熊秦邱江尹薛闫段雷侯龙史黎贺顾毛郝龚邵万钱覃武戴孔汤庞樊兰殷施陶洪翟安颜倪严牛温芦季俞章鲁葛伍申尤毕聂柴焦向柳邢岳齐沿梅莫庄辛管祝左涂谷祁时舒耿牟卜路詹关苗凌费纪靳盛童欧甄项曲成游阳裴席卫查屈鲍位覃霍翁隋植甘景薄单包司柏宁柯阮桂闵阳解强丛华车冉房边辜吉饶刁瞿戚丘古米池滕晋苑邬臧畅宫来嵺苟全褚廉简娄盖符奚木穆党燕郎邸冀谈姬屠连郜晏栾郁商蒙计喻揭窦迟宇敖糜鄢冷卓花艾蓝都巩稽井练仲乐虞卞封竺冼原官衣楚佟栗匡宗应台巫鞠僧桑荆谌银扬明沙薄伏岑习胥保和蔺水云昌凤酆常皮康元平萧湛禹无贝茅麻危骆支咎经裘缪干宣贲杭诸钮嵇滑荣荀羊於惠家芮羿储汲邴松富乌巴弓牧隗山宓蓬郗班仰秋伊仇暴钭厉戎祖束幸韶蓟印宿怀蒲鄂索咸籍赖乔阴能苍双闻莘贡逢扶堵宰郦雍却璩濮寿通扈郏浦尚农别阎充慕茹宦鱼容易慎戈庚终暨居衡步满弘国文寇广禄阙东殴殳沃利蔚越夔隆师厍晃勾融訾阚那空毋乜养须丰巢蒯相后红逯益桓公俟马官侯葛人方连甫迟羊台冶政阳于于叔屠孙孙辕狐离文孙容于丘徒空官寇仉督车孙木马西雕正驷良跋谷父粱法汝钦干里郭门延海舌生帅缑亢况郈琴丘丘门门佘佴伯赏宫墨哈谯笪年爱仝代]'
xin2 = '欧阳|权逯|盖益|万俟|司马|上官|夏侯|诸葛|闻人|东方|赫连|皇甫|尉迟|公羊|澹台|公冶|宗政|濮阳|淳于|单于|太叔|申屠|公孙|仲孙|轩辕|令狐|钟离|宇文|长孙|慕容|鲜于|闾丘|司徒|司空|亓官|司寇|子车|颛孙|端木|巫马|公西|漆雕|乐正|壤驷|公良|拓跋|夹谷|宰父|谷粱|段干|百里|东郭|南门|呼延|归海|羊舌|微生|梁丘|左丘|东门|西门|南宫'
regex = f"(?<![一-龟])(?:{xin1}|{xin2})[\u4E00-\u9FA5]{{1,2}}"
# re.findall(r'\b[\u4e00-\u9fa5]{2,3}\b',s)
re.findall(regex, s)


> pd.Series.str.get_dummies(sep=)  
    > 根据分隔符分开，dummy化（去重，有就用1表示，没有就用0表示），并返回df

In [ ]:
pd.Series(['a,a,b', 'cd,a', 'b,cd,cd']).str.get_dummies(',')


> feather

In [ ]:

df = pd.DataFrame({'a': [1, 2, 3]})
df.to_feather('temp.feather')


In [ ]:
from pyarrow import feather
feather.write_feather(df, 'temp.feather')


> reindex   
> * 重新排序，没有字段添加默认值

In [ ]:
df = pd.DataFrame(np.random.random(size=(3, 3)), columns=list('abc'))
df.reindex(list('dcba'), axis=1, fill_value=0)


> 追加写入excel

In [ ]:
df1 = pd.DataFrame(['1', '2'])
for i in range(3):
    with pd.ExcelWriter('temp.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as f:
        df1.to_excel(f, sheet_name=f'{i}')


> 修饰器

In [ ]:
import time
from functools import wraps


def xsq(f):

    @wraps(f)
    def wrap(*args, **kwargs):
        start_t = time.perf_counter()
        r = f(*args, **kwargs)
        print(f'用时:{(time.perf_counter()-start_t):.2f}秒')
        return r
    return wrap


@xsq
def myf(x):
    print(x)
    time.sleep(1)
    return x*3


abc = myf(34)


In [ ]:
# xw.Range('a1').value=np.random.randint(4,size=(10,10))
df = xw.Range('a1').current_region.options('df', index=0,).value
df
df.groupby('Date', as_index=False).apply(
    lambda x: x.agg(lambda x: x.dropna().unique())).droplevel(1)
df.groupby('Date', as_index=False).apply(lambda x: x.bfill().iloc[0])


In [ ]:
class A():
    def __init__(self):
        self.name1 = 'A'

    def show(self):
        print(self.name1)


class B(A):
    def __init__(self):
        super(A).__init__()
        self.name1 = 'B'


a = A()
b = B()
b.show()
a.show()


In [ ]:
from collections import namedtuple

Stock = namedtuple('Stock', ['name', 'shares', 'price', 'date', 'time'])

# Create a prototype instance
stock_prototype = Stock('', 0, 0.0, None, None)

# Function to convert a dictionary to a Stock


def dict_to_stock(s):
    return stock_prototype._replace(**s)


#### 数据库 sqllite3

In [ ]:
import sqlite3

conn = sqlite3.connect('test1.db')
print("数据库打开成功")
c = conn.cursor()
c.execute('''CREATE TABLE COMPANY
       (姓名          TEXT     NOT NULL,
       学科           TEXT    NOT NULL,
       分数            INT     NOT NULL
       );''')
print("数据表创建成功")
conn.commit()
conn.close()


In [ ]:
import sqlite3

conn = sqlite3.connect('test1.db')
c = conn.cursor()
print("数据库打开成功")

c.execute("INSERT INTO COMPANY (姓名,学科,分数) \
      VALUES ('张三', '语文', 81)")
c.execute("INSERT INTO COMPANY (姓名,学科,分数) \
      VALUES ('张三', '数学', 75)")
c.execute("INSERT INTO COMPANY (姓名,学科,分数) \
      VALUES ('李四', '语文', 76)")
c.execute("INSERT INTO COMPANY (姓名,学科,分数) \
      VALUES ('李四', '数学', 90)")
c.execute("INSERT INTO COMPANY (姓名,学科,分数) \
      VALUES ('王五', '语文', 81)")
c.execute("INSERT INTO COMPANY (姓名,学科,分数) \
      VALUES ('王五', '数学', 100)")


conn.commit()
print("数据插入成功")
conn.close()


In [ ]:
import sqlite3

conn = sqlite3.connect('test1.db')
c = conn.cursor()
print("数据库打开成功")

cursor = c.execute("SELECT *  from COMPANY")
for eh in (cursor):
    print(eh, type(eh))

conn.close()


#### 使用mysql

> 利用pandas的index 生成不同的数据类型 生成批量写入mysql

In [ ]:
df = pd.DataFrame({'a': [1, 2, 1], 'b': [3, 4., 3], 'c': list('abc')})
df['a'] = df.a.astype(int)
df
df = df.set_index([*df])
sqldata = df.index
sql = "insert into mytable ('a','b','c') values " + ','.join(map(str, sqldata))
# mysql批量写入
sql


In [ ]:
from connmysql import QQ_Sql as Q
with Q() as q:
    r = q.execute('show tables; ')
r


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus as urlquote  # 为了解决密码含有@字符导致engine无法连接数据库
para = dict(
    user='Numpy_Pandas_admin',
    password=urlquote('Npandas123!@#'),
    host='39.101.131.154',
    database='Python_Numpy_Pandas_QQ',
    charset='utf8', )
_db_str = 'mysql+pymysql://{user}:{password}@{host}:3306/{database}?charset={charset}'.format(
    **para)
conn = create_engine(_db_str)
pd.read_sql('select * from ayitemp', con=conn)


#### 图片播放背景音乐告白

In [ ]:
import random
import pygame as py
import tkinter as tk
from time import time, sleep
from tkinter import filedialog
from PIL import Image, ImageTk
from math import sin, cos, radians
from random import choice, uniform, randint


class part:
    # 为每一个烟花绽放出来的粒子单独构建一个类的对象 ，每个粒子都会有一些重要的属性，决定它的外观（大小、颜色）、移动速度等
    def __init__(self, cv, idx, total, explosion_speed, x=0., y=0., vx=0., vy=0., size=2., color='red', lifespan=2, **kwargs):
        self.id = idx
        # 每个烟花的特定标识符
        self.x = x
        # 烟花绽放x轴
        self.y = y
        # 烟花绽放y轴
        self.initial_speed = explosion_speed
        # 粒子初始速度
        self.vx = vx
        # 粒子运动x轴速度
        self.vy = vy
        # 粒子运动y轴速度
        self.total = total
        # 绽放粒子数
        self.age = 0
        # 粒子已停留时间
        self.color = color
        # 粒子颜色
        self.cv = cv
        # 画布
        self.cid = self.cv.create_oval(
            x - size, y - size, x + size, y + size, fill=self.color, outline='white', width=0.01)
        # 指定一个限定矩形（Tkinter 会自动在这个矩形内绘制一个椭圆）
        self.lifespan = lifespan
        # 粒子在画布上停留的时间

    def update(self, dt):
        self.age += dt
        # 更新粒子停留时间
        if self.alive() and self.expand():
            # 如果粒子既存活又处于扩张阶段
            move_x = cos(radians(self.id*360/self.total))*self.initial_speed
            # 粒子x轴继续膨胀
            move_y = sin(radians(self.id*360/self.total))*self.initial_speed
            # 粒子y轴继续膨胀
            self.cv.move(self.cid, move_x, move_y)
            # 根据id把画布上的粒子移动x和y个距离
            self.vx = move_x/(float(dt)*1000)
            # 粒子x轴的速度

        elif self.alive():
            columnFont = ('华文行楷', 14)
            # 如果粒子仅存活不扩张（只是停留时间足够，说明膨胀到最大了），则自由坠落
            self.cv.create_text(250, 100, text='喜', tag="write_tag", fill=choice(
                colors), font=columnFont)  # 字体
            self.cv.create_text(
                300, 100,  text='欢', tag="write_tag", fill=choice(colors), font=columnFont)
            self.cv.create_text(350, 100, text='你', tag="write_tag",
                                fill=choice(colors), font=columnFont)
            self.cv.create_text(
                400, 100,  text='吖', tag="write_tag", fill=choice(colors), font=columnFont)
            # 删除文字标签
            move_x = cos(radians(self.id*360/self.total))
            # x轴的移动位移
            # we technically don't need to update x, y because move will do the job
            self.cv.move(self.cid, self.vx + move_x, self.vy+GRAVITY*dt)
            self.vy += GRAVITY*dt
            # 更新y轴

        elif self.cid is not None:
            # 如果粒子生命周期已过，则将其移除
            cv.delete(self.cid)
            # 在画布上移除该粒子对象
            self.cv.delete("write_tag")
            # 同时移除字体
            self.cid = None

    def expand(self):
        # 定义膨胀效果时间帧
        return self.age <= 1.2
        # 判断膨胀时间是否小于1.2秒

    def alive(self):
        # 判断粒子是否仍在生命周期内
        return self.age <= self.lifespan
        # 判断已停留时间是否小于应该停留时间


'''
Firework simulation loop:
Recursively call to repeatedly emit new fireworks on canvas
a list of list (list of stars, each of which is a list of particles)
is created and drawn on canvas at every call, 
via update protocol inside each 'part' object 
'''


def randomcolor():
    # 生成随机颜色
    colArr = ['1', '2', '3', '4', '5', '6', '7',
              '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
    color = ""
    for i in range(6):
        color += colArr[random.randint(0, 14)]
    return "#"+color


GRAVITY = 0.06
# 重力变量
colors = ['red', 'blue', 'yellow', 'white', 'green', 'orange',
          'purple', 'seagreen', 'indigo', 'cornflowerblue', 'pink']
# 颜色列表


def simulate(cv):

    t = time()
    # 返回自1970年后经过的浮点秒数，精确到小数点后7位
    explode_points = []
    # 爆炸点列表，烟花列表
    wait_time = randint(10, 100)
    # 等待时间为10到100之间整数
    numb_explode = randint(8, 20)
    # 爆炸烟花个数时6到10之间的随机整数
    # create list of list of all particles in all simultaneous explosion
    for point in range(numb_explode):
        # 为所有模拟烟花绽放的全部粒子创建一列列表
        if point <= 4:
            objects = []
            # 每个点的爆炸粒子列表粒子列表
            x_cordi = 250 + point*50
            # 每个爆炸点的x轴
            y_cordi = 100
            # 每个爆炸点的y轴
            speed = uniform(0.5, 1.5)
            # 每个爆炸点的速度
            size = uniform(0.5, 3)
            # 每个爆炸点的大小
            color = choice(colors)
            # 每个爆炸点的颜色
            explosion_speed = uniform(0.6, 3)
            # 爆炸的绽放速度
            total_particles = randint(10, 60)
            # 烟花的总粒子数
            for i in range(1, total_particles):
                # 同一个烟花爆炸出来的粒子大小、速度、坐标都是相同的
                r = part(cv, idx=i, total=total_particles, explosion_speed=explosion_speed, x=x_cordi,
                         y=y_cordi, vx=speed, vy=speed, color=color, size=size, lifespan=uniform(0.6, 1.75))
                # 把上述参数代入part函数，但是每个粒子的生存时间是自己独立的
                objects.append(r)
                # 把r添加进粒子列表
            explode_points.append(objects)
            # 把粒子列表添加进烟花列表
        else:
            objects = []
            # 每个点的爆炸粒子列表粒子列表
            x_cordi = randint(50, 550)
            # 每个爆炸点的x轴
            y_cordi = randint(50, 150)
            # 每个爆炸点的y轴
            speed = uniform(0.5, 1.5)
            # 每个爆炸点的速度
            size = uniform(0.5, 3)
            # 每个爆炸点的大小
            color = choice(colors)
            # 每个爆炸点的颜色
            explosion_speed = uniform(0.3, 2)
            # 爆炸的绽放速度
            total_particles = randint(10, 50)
            # 烟花的总粒子数
            for i in range(1, total_particles):
                # 同一个烟花爆炸出来的粒子大小、速度、坐标都是相同的
                r = part(cv, idx=i, total=total_particles, explosion_speed=explosion_speed, x=x_cordi,
                         y=y_cordi, vx=speed, vy=speed, color=color, size=size, lifespan=uniform(0.6, 1.75))
                # 把上述参数代入part函数，但是每个粒子的生存时间是自己独立的
                objects.append(r)
                # 把r添加进粒子列表
            explode_points.append(objects)
            # 把粒子列表添加进烟花列表

    total_time = .0
    # 初始化总时间
    # keeps undate within a timeframe of 1.8 second
    while total_time < 2:
        # 当总时间小于1.8秒时运行该循环
        sleep(0.03)
        # 让画面暂停0.01秒
        tnew = time()
        # 刷新时间
        t, dt = tnew, tnew - t
        # 时间等于新时间，和上次时间间隔为tnew-t
        for point in explode_points:
            # 遍历烟花列表
            for item in point:
                # 遍历烟花里的粒子列表
                item.update(dt)
                # 粒子更新时间
        cv.update()
        # 刷新画布
        total_time += dt
        # 为while循环增加时间

    root.after(wait_time, simulate, cv)
    # 将组件置于其他组件之后，放在最顶层，覆盖下面的，递归调用自己，形成新一轮的爆炸


def close(*ignore):
    # 打开模拟循环并关闭窗口
    """Stops simulation loop and closes the window."""
    global root
    root.quit()


if __name__ == '__main__':
    root = tk.Tk()
    root.title('漫天烟花——祝大家—有情人终成眷属')  # 设置窗体的标题栏
    cv = tk.Canvas(root, height=600, width=600)
    # 绘制一个高600，宽600的画布
    bgpath = filedialog.askopenfilename(title='请选择背景图片')
    # 选择背景图片
    image = Image.open(bgpath)
    # 打开背景图片
    image = image.resize((600, 600), Image.ANTIALIAS)
    # 把背景图片调整成窗口大小
    photo = ImageTk.PhotoImage(image)
    cv.create_image(0, 0, image=photo, anchor='nw')
    # 在画布上绘制加载的背景图片
    bgmusic = filedialog.askopenfilename(title='请选择背景音乐')
    py.mixer.init()
    # 初始化
    py.mixer.music.load(bgmusic)
    # 文件加载
    py.mixer.music.play(-1, 0, fade_ms=50)
    # 播放  第一个是播放值 -1代表循环播放， 第二个参数代表开始播放的时间
    py.mixer.music.pause()
    # 暂停
    py.mixer.music.unpause()
    # 取消暂停
    cv.pack()
    # 把cv添加进去
    root.protocol("WM_DELETE_WINDOW", close)
    root.after(200, simulate, cv)
    # 在0.1秒后再调用stimulate函数，生成一轮烟花绽放效果
    root.mainloop()
    # 执行root，生成窗口


#### excel查找图片
根据图片的左边单元格的值作为key进行匹配
> 用法:
>   1. 打开要处理的excel表。
>   2. 图片所在的表赋值给sht_source
>   3. 目标查找的表赋值给sht_sht_target，结果以a列进行匹配

In [ ]:
import xlwings as xw


def get_pic(sht_source):
    '''获取源表所有图片'''
    sh = sht_source.shapes
    sh = [x for x in sh if x.type == "picture"]  # 仅选取图片类型
    shd = {sht_source.range(x.api.TopLeftCell.Row, x.api.TopLeftCell.Column - 1
                            ).value: x for x in sh}  # 转字典并且用前一行的值作为key
    return shd


def put_pic(sht_target, shd):
    '''写入图片'''
    sht_target.activate()
    for eh in sht_target.used_range.columns[0]:
        if shd.get(eh.value):
            shd[eh.value].api.Copy()
            eh.offset(0, 1).select()
            sht_target.api.Paste()


if __name__ == '__main__':
    wb = xw.books.active
    sht_source = wb.sheets[1]  # 图片所在表
    sht_target = wb.sheets[0]  # 要写入的表，以a列查找，图片放到a列是右边一列
    put_pic(sht_target, get_pic(sht_source))


#### office文件转pdf  
> excel仅仅转第一个sheet,其他office也是转单个文件

In [ ]:
# -*- coding:utf-8 -*-
import os
from win32com.client import Dispatch, constants, gencache, DispatchEx


class PDFConverter:
    def __init__(self, pathname, export='.'):
        self._handle_postfix = ['doc', 'docx', 'ppt', 'pptx', 'xls', 'xlsx']
        self._filename_list = []
        self._export_folder = os.path.join(os.path.abspath('.'), 'pdfconver')
        if not os.path.exists(self._export_folder):
            os.mkdir(self._export_folder)
        self._enumerate_filename(pathname)

    def _enumerate_filename(self, pathname):
        """
        读取所有文件名
        """
        full_pathname = os.path.abspath(pathname)
        if os.path.isfile(full_pathname):
            if self._is_legal_postfix(full_pathname):
                self._filename_list.append(full_pathname)
            else:
                raise TypeError(
                    f"文件 {pathname} 后缀名不合法！仅支持如下文件类型：{'、'.join(self._handle_postfix)}。")

        elif os.path.isdir(full_pathname):
            for relpath, _, files in os.walk(full_pathname):
                for name in files:
                    filename = os.path.join(full_pathname, relpath, name)
                    if self._is_legal_postfix(filename):
                        self._filename_list.append(os.path.join(filename))
        else:
            raise TypeError(f'文件/文件夹 {pathname} 不存在或不合法！')

    def _is_legal_postfix(self, filename):
        return filename.split('.')[-1].lower() in self._handle_postfix and not os.path.basename(filename).startswith('~')

    def run_conver(self):
        '''
        进行批量处理，根据后缀名调用函数执行转换
        '''
        print('需要转换的文件数：', len(self._filename_list))
        for filename in self._filename_list:
            postfix = filename.split('.')[-1].lower()
            funcCall = getattr(self, postfix)
            print('原文件：', filename)
            funcCall(filename)
        print('转换完成！')

    def doc(self, filename):
        '''
        doc 和 docx 文件转换
        '''
        name = os.path.basename(filename).split('.')[0] + '.pdf'
        exportfile = os.path.join(self._export_folder, name)
        print('保存 PDF 文件：', exportfile)
        gencache.EnsureModule(
            '{00020905-0000-0000-C000-000000000046}', 0, 8, 4)
        w = Dispatch("Word.Application")
        doc = w.Documents.Open(filename)
        doc.ExportAsFixedFormat(exportfile, constants.wdExportFormatPDF,
                                Item=constants.wdExportDocumentWithMarkup,
                                CreateBookmarks=constants.wdExportCreateHeadingBookmarks)

        w.Quit(constants.wdDoNotSaveChanges)

    def docx(self, filename):
        self.doc(filename)

    def xls(self, filename):
        '''
        xls 和 xlsx 文件转换
        '''
        name = os.path.basename(filename).split('.')[0] + '.pdf'
        exportfile = os.path.join(self._export_folder, name)
        xlApp = DispatchEx("Excel.Application")
        xlApp.Visible = False
        xlApp.DisplayAlerts = 0
        books = xlApp.Workbooks.Open(filename, False)
        books.ExportAsFixedFormat(0, exportfile)
        books.Close(False)
        print('保存 PDF 文件：', exportfile)
        xlApp.Quit()

    def xlsx(self, filename):
        self.xls(filename)

    def ppt(self, filename):
        '''
        ppt 和 pptx 文件转换
        '''
        name = os.path.basename(filename).split('.')[0] + '.pdf'
        exportfile = os.path.join(self._export_folder, name)
        gencache.EnsureModule(
            '{00020905-0000-0000-C000-000000000046}', 0, 8, 4)
        p = Dispatch("PowerPoint.Application")
        ppt = p.Presentations.Open(filename, False, False, False)
        ppt.ExportAsFixedFormat(exportfile, 2, PrintRange=None)
        print('保存 PDF 文件：', exportfile)
        p.Quit()

    def pptx(self, filename):
        self.ppt(filename)


if __name__ == "__main__":

    # 支持文件夹批量导入
    # folder = 'tmp'
    # pathname = os.path.join(os.path.abspath('.'), folder)

    # 也支持单个文件的转换
    pathname = 'temp.docx'

    pdfConverter = PDFConverter(pathname)
    pdfConverter.run_conver()


> excel按照sheet名称转化为pdf

In [ ]:
# -*- coding:utf-8 -*-

import os
from win32com.client import Dispatch, constants, gencache, DispatchEx
import xlrd
import openpyxl


class PDFConverter:
    def __init__(self, pathname, sheetnum, export='.'):
        self.sheetnum = sheetnum
        self._handle_postfix = ['xls', 'xlsx']
        self._filename_list = []
        self._export_folder = os.path.join(os.path.abspath('.'), 'pdfconver')
        if not os.path.exists(self._export_folder):
            os.mkdir(self._export_folder)
        self._enumerate_filename(pathname)

    def _enumerate_filename(self, pathname):
        '''
        读取所有文件名
        '''
        full_pathname = os.path.abspath(pathname)
        if os.path.isfile(full_pathname):
            if self._is_legal_postfix(full_pathname):
                self._filename_list.append(full_pathname)
            else:
                raise TypeError(
                    f"文件 {pathname} 后缀名不合法！仅支持如下文件类型：{'、'.join(self._handle_postfix)}。")

        elif os.path.isdir(full_pathname):
            for relpath, _, files in os.walk(full_pathname):
                for name in files:
                    filename = os.path.join(full_pathname, relpath, name)
                    if self._is_legal_postfix(filename):
                        self._filename_list.append(os.path.join(filename))
        else:
            raise TypeError(f'文件/文件夹 {pathname} 不存在或不合法！')

    def _is_legal_postfix(self, filename):
        return filename.split('.')[-1].lower() in self._handle_postfix and not os.path.basename(filename).startswith(
            '~')

    def run_conver(self):
        '''
        进行批量处理，根据后缀名调用函数执行转换
        '''
        print('需要转换的文件数：', len(self._filename_list))
        for filename in self._filename_list:
            postfix = filename.split('.')[-1].lower()
            funcCall = getattr(self, postfix)
            print('原文件：', filename)
            funcCall(filename)
        print('转换完成！')

    def xls(self, filename):
        '''
        xls 和 xlsx 文件转换
        '''
        xlApp = DispatchEx("Excel.Application")
        xlApp.Visible = False
        xlApp.DisplayAlerts = 0
        books = xlApp.Workbooks.Open(filename, False)
        # 循环保存每一个sheet
        for i in range(1, self.sheetnum+1):
            sheetName = books.Sheets(i).Name
            xlSheet = books.Worksheets(sheetName)
            name = f'{sheetName}.pdf'
            exportfile = os.path.join(self._export_folder, name)
            xlSheet.ExportAsFixedFormat(0, exportfile)
            print('保存 PDF 文件：', exportfile)
        books.Close(False)
        xlApp.Quit()

    def xlsx(self, filename):
        self.xls(filename)


if __name__ == "__main__":
    # 支持单个文件的转换
    pathname = 'temp.xls'
    # 获取到文件的sheet数

    if pathname.split('.')[-1] == 'xls':
        sheetnum = xlrd.open_workbook(pathname).sheets().__len__()
    else:
        sheetnum = openpyxl.load_workbook(pathname).sheetnames.__len__()
    pdfConverter = PDFConverter(pathname, sheetnum)
    pdfConverter.run_conver()


#### 临时练习

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.array(["Runoob-1", "Runoob-2", "Runoob-3", "C-RUNOOB"])
y = np.random.randint(10, size=(4,))

plt.bar(x, y,  color=["#4CAF50", "red", "hotpink", "#556B2F"])
plt.show()


In [ ]:
import pandas as pd
import xlwings as xw
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
mysize = 20
np.random.seed(1)
x = np.arange(mysize)
y = np.random.randint(1, 10, size=(mysize,))
mycolor = ["red", "yellow",  "green", "blue"]
mycolor = (pd.cut(y, len(mycolor), labels=mycolor))
mycolor.value_counts()
plt.bar(x, y,  color=mycolor)
plt.show()
# mycolor



In [ ]:
import tushare as ts
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


def getdata(code):
    # return ts.get_hist_data(code, start='2022-07-01', end='2022-12-31', ktype="d",).assign(code=code)
    rs = ts.get_hist_data(code, start='2022-07-01',
                          end='2022-12-31', ktype="d",)
    return pd.DataFrame() if rs is None else rs[['close']].set_axis([code], axis=1)


codelist = ['600000', '600008', '833943',
            '000112', '430090', '430418']
with ThreadPoolExecutor(max_workers=5) as executor:
    alldata = executor.map(getdata, codelist)
result = pd.concat(list(alldata), axis=1)
result


In [ ]:
import pandas as pd
import numpy as np


filename = '济南市.xlsx'
bins = [0, 20, 50, 80, 100, 130, 150, np.inf]
labels = '''20元以下
20-50元
50-80元
80-100元
100-130元
130-150元
150元以上
'''.splitlines()

with pd.ExcelWriter(filename, mode='a', engine='openpyxl', if_sheet_exists='replace') as f:
    df = pd.read_excel(f)
    # dfr=df.groupby(pd.cut(df.价格,bins=bins,labels=labels))['店铺名称'].count().to_frame('店铺数量')
    # dfr.to_excel(f,sheet_name='结果')


In [ ]:
df = xw.Range('a1').current_region.options(
    'df', index=False, numbers=int).value
df
df.nlargest(5, 'b', 'all')


In [ ]:
from itertools import zip_longest


def f(se):
    return list(zip_longest(*se))


df = pd.read_json('info.json')
col = df.columns[:3].to_list()

(df.replace({'': pd.NA})
    .groupby(col)
    .agg(lambda x: x.dropna().values)
    .apply(f, axis=1)
    .explode().apply(pd.Series)
    .set_axis(df.columns[3:], axis=1)
    .set_index('high_community', append=True))


In [7]:
import pandas as pd
from itertools import zip_longest
from chinese_province_city_area_mapper.infrastructure import SuperMap
from chinese_province_city_area_mapper import drawers


def getdata(filename):
    df = pd.read_json(filename)
    col = df.columns[:3].to_list()
    return (df.replace({'': pd.NA})
            .groupby(col)
            .agg(lambda x: x.dropna().values)
            .apply(lambda x: list(zip_longest(*x)), axis=1)
            .explode().apply(pd.Series)
            .set_axis(df.columns[3:], axis=1)
            )


df = getdata('info.json')
df = df.reset_index().iloc[:, :3].set_axis('省 市 区'.split(), axis=1)
df["经纬度"] = df.apply(
    lambda x: SuperMap.lat_lon_mapper.get(",".join(x)), axis=1)
drawers.draw_locations(df, "df.html")


In [ ]:
from docxtpl import DocxTemplate
template = DocxTemplate('jinjatemp.docx')
data = {"A": "192.168.1.11", "B": ['192.168.1.1', '192.168.1.2']}
template.render(data)
template.save('test.docx')


In [ ]:
'''右键添加菜单'''
import win32api
import win32con

key = win32api.RegOpenKey(win32con.HKEY_LOCAL_MACHINE,
                          r"SOFTWARE\Classes\*\shell")

newKey = win32api.RegCreateKey(key, "YNote")

sub_key = win32api.RegOpenKey(
    win32con.HKEY_LOCAL_MACHINE, r"SOFTWARE\Classes\*\shell\YNote")

newsubKey = win32api.RegCreateKey(sub_key, "command")

win32api.RegSetValue(newsubKey, win32con.REG_SZ,
                     "\"C:\Program Files (x86)\Youdao\YoudaoNote\YoudaoNote.exe\" \"%1\"")


In [ ]:
from urllib.parse import urlencode
alist = [{'a': 'aaa', 'b': 'bbb', 'c': 1.25}]
next(map(urlencode, alist)).replace('&', ';')


In [ ]:
np.tril(np.arange(8))
np.tile(np.arange(9).reshape(3, 3), (2, 2))


In [ ]:
n = 10
[bin(i)[2:].zfill(n.bit_length()) for i in range(n)]
[f'{i:0{n.bit_length()}b}' for i in range(n)]


In [ ]:
import json
import pandas as pd
import xlwings as xw
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

s = '2022-09-13T09:09:19+08:00'
pd.to_datetime(s).tz_localize(None)


In [ ]:
from pandas.tseries.offsets import DateOffset
df = xw.Range('a1').current_region.options('df', index=False).value
df['date'] = pd.to_datetime(df['date'].astype(int).astype(str))
df['ndate'] = df['date'] + DateOffset(months=1)
df['max'] = df.resample('Q', on='ndate')['value'].transform('max')
df[df.ndate.dt.quarter == 1]


In [ ]:
path = r'D:\百度云同步盘\工作\2021\酬金\isee系统合同号级清单'
filename = '2021{:02}.xls'
filename = os.path.join(path, filename)
alist = []
for i in range(1, 11):
    ehf = filename.format(i)
    alist.append(pd.read_excel(ehf, header=2).assign(月份=i))
dfall = pd.concat(alist)
dfall


In [ ]:
df = xw.Range('a1').current_region.options(
    'df', index=False, numbers=int).value
sp_p = 2 # 分割的列
col_exp = df.columns[sp_p:]
n = len(col_exp)
df1 = (df.groupby(df.columns[:sp_p].to_list(), sort=False)
       .apply(lambda x: x[col_exp].values.ravel())
       .apply(pd.Series))
df1.columns = df1.columns.map(
    lambda x: np.choose(x % n, col_exp) + str(x//n+1))



In [ ]:
df = xw.Range('a1').current_region.options(
    'df', index=False, numbers=int).value

df
df.apply(lambda x:x
            .set_axis(x.str.extract(r'(\D+)')[0])
            .dropna()
            .reindex(df.columns,)
         , axis=1)


In [ ]:
import pandas as pd
import xlwings as xw
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from itertools import accumulate
list(accumulate(pd.Series([100,10,1]),func=lambda x,y:np.mean([x,y])))
